In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

import pandas as pd
import numpy as np

from create_staging_tables import create_staging_merged_initial_df
from web_scrape_functions import scrape_bill_url

In [ ]:
engine = create_engine('postgresql://localhost:5432/wa_leg_staging')

In [ ]:
con = engine.connect()

In [ ]:
merged_initial_df = pd.read_sql_query('select * from "merged_initial"',con=engine)

In [ ]:
bill_text_df = pd.read_sql_query('select * from "bill_text"',con=engine)

In [ ]:
legislator_df = pd.read_sql_query('select * from "legislator"',con=engine)

In [ ]:
merged_initial_df.info()

In [ ]:
legislator_df

In [ ]:
bill_text_df.shape

In [ ]:
merged_second = merged_initial_df.merge(bill_text_df, how='left', on=['unique_id', 'htm_url'])

In [ ]:
merged_second

In [ ]:
def change_agency_to_int(agency):
    if agency == 'House':
        return 0
    else:
        return 1

In [ ]:
legislator_df['agency'] = legislator_df['agency'].apply(change_agency_to_int)

In [ ]:
merged_final = merged_second.merge(legislator_df, how='left', left_on=['voter_id', 'voting_agency'], right_on=['id', 'agency'])

In [ ]:
def create_staging_merged_final_df(merged_initial_df, bill_text_df, legislator_df):
    '''Create merged_final pandas dataframe. This dataframe contains all necessary data and is ready for 
    cleaning.
    Input
    merged_initial_df: pandas dataframe loaded from wa_leg_staging database, merged_inital table
    bill_text_df: pandas dataframe loaded from wa_leg_staging database, bill_text table
    legislator_df: pandas dataframe loaded from wa_leg_staging database, legislator table
    '''
    merged_second_df = merged_initial_df.merge(bill_text_df, how='left', on=['unique_id', 'htm_url'])
    
    def change_agency_to_int(agency):
        if agency == 'House':
            return 0
        if agency == 'Senate':
            return 1
    
    legislator_df['agency'] = legislator_df['agency'].apply(change_agency_to_int)
    merged_final = merged_second_df.merge(legislator_df, 
                               how='left', 
                               left_on=['voter_id', 'voting_agency'], 
                               right_on=['id', 'agency'])
    
    return merged_final

In [ ]:
MERGED_final = create_staging_merged_final_df(merged_initial_df, bill_text_df, legislator_df)